In [1]:
import torch
import torch_snippets

C:\Users\olive\miniconda3\envs\torch_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet

C:\Users\olive\OneDrive - University of Southampton\Research\Object Detection\RCNN\darknet


fatal: destination path 'darknet' already exists and is not an empty directory.


## Reconfigure the Makefile 

In [3]:
# sed is replacing opencv=0 with opencv=1 in the darknet makefile 
# this is forcing it to install with opencv 

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile

if torch.cuda.is_available():
    !sed -i 's/GPU=0/GPU=1' Makefile
    !sed -i 's/CUDNN=0/CUDNN=1/' Makefile
    !sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

'sed' is not recognized as an internal or external command,
operable program or batch file.


## Compile the Makefile 

In [11]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp:253:21: warning: unused variable 'n' [-Wunused-variable]
                int n = _write(client, outputbuf, outlen);
                    ^
1 warning generated.
gcc -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c:2044:15: warning: unused variable 'out_w' [-Wunused-variable]
    const int out_w = (w + 2 * pad - ksize) / stride + 1;    // output_width=input_width for stride=1 and pad=1
              ^
./src/gemm.c:2043:15: warning: unused variable 'out_h' [-Wunused-v

## Download the dataset

In [12]:
!wget --quiet https://www.dropbox.com/s/agmzwk95v96ihic/open-images-bus-trucks.tar.xz
!tar -xf open-images-bus-trucks.tar.xz
!rm open-images-bus-trucks.tar.xz

## Download YOLO Weights

In [ ]:
!wget --quiet https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

## Check it is installed

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/person.jpg

# Setting up the dataset format

In [ ]:
%%writefile data/obj.names
bus
truck

In [ ]:
%%writefile data/obj.data
classes = 2
train = data/train.txt
valid = data/val.txt
names = data/obj.names
backup = backup/

In [ ]:
# move the images and ground truth text files to data/obj folder 

# Make the data directory
!mkdir -p data/obj

In [ ]:
# copy the images to the data/obj directory
!cp -r open-images-bus-trucks/images/* data/obj/

In [ ]:
# move the train and text labels to data/
!cp -r open-images-bus-trucks/yolo_labels/all/{train,val}.txt data/

In [ ]:
# move the 
!find ./open-images-bus-trucks/yolo_labels/all/labels -name "*.txt" -exec cp {} ./data/obj/ \;

In [ ]:
# Each of the labels files is a text file with structure [cls, xc, yc, w, h]

# Configuring the Network

In [ ]:
# create a copy of existing configuration and modify it in place
!cp cfg/yolov4-tiny-custom.cfg cfg/yolov4-tiny-bus-trucks.cfg

# max_batches to 4000 (since the dataset is small enough)
!sed -i 's/max_batches = 500200/max_batches=4000/' cfg/yolov4-tiny-bus-trucks.cfg

# number of sub-batches per batch
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov4-tiny-bus-trucks.cfg

# number of batches after which learning rate is decayed
!sed -i 's/steps=400000,450000/steps=3200,3600/' cfg/yolov4-tiny-bus-trucks.cfg

# number of classes is 2 as opposed to 80 (which is the number of COCO classes)
!sed -i 's/classes=80/classes=2/g' cfg/yolov4-tiny-bus-trucks.cfg

# in the classification and regression heads, change number of output convolution filters
# from 255 -> 21 and 57 -> 33, since we have fewer classes we don't need as many filters
!sed -i 's/filters=255/filters=21/g' cfg/yolov4-tiny-bus-trucks.cfg
!sed -i 's/filters=57/filters=33/g' cfg/yolov4-tiny-bus-trucks.cfg

# Training the Model

In [ ]:
# Download the model into the weights then and load them into the model 
!wget --quiet https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
    
!cp yolov4-tiny.conv.29 build/darknet/x64

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-tiny-bus-trucks.cfg yolov4-tiny.conv.29 -dont_show -mapLastAt

# Evaluation

In [ ]:
from torch_snippets import Glob, stem, show, read
import glob
import os 
%cd darknet

root = os.getcwd()

img = "/Users/olivergrainge/Documents/RCNN/darknet/test_images/test_img.jpg"
!sed -i 's/classes=80/classes=2/g' cfg/yolov4-tiny-bus-trucks.cfg


print(os.getcwd())
!./darknet detector test \
data/obj.data cfg/yolov4-tiny-bus-trucks.cfg\
backup/yolov4-tiny-bus-trucks_last.weights {img}

    
#./darknet detector test cfg/coco.data cfg/yolov3.cfg yolov3.weights data/dog.jpg